<h1>Intruder Detection using machine Learning</h1>
<h3>Distinction to detect intruder</h3>
<ul>
<li>Mask Detection : Detects if person entering premises is wearing any mask or not</li>
<li>Weapon Detection : Detects if person entering premises is carying any gun or not</li>
<li>Authorisation  : Detects if person entering premises is known or not </li>
</ul>

<h3>Installing and importing dependencies</h3>

In [1]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

<h3>Collecting image data</h3>
<p>The data will be in form of urls and paths which will be stored in a python dictionary structure. </p>
<p>Note: The data must be uploaded on Google Drive and access permission must be given in case using the same data as below code </p>
<p>The data can also is be stored in local folder , in that case relative paths must be added</p>
<p style="color:red">Note : Following images are from local folder so they are not included in this code</p>

<h4>Below map contains training data for masked model</h4>
<p>It includes various images of masked people from different angles and in different conditions</p>

In [ ]:
data = {
    "https://media.istockphoto.com/id/1272058180/photo/portrait-of-a-young-man-wearing-protective-face-mask.webp?b=1&s=170667a&w=0&k=20&c=dRWUCyd0VtQg6P15-HsiSlqE1dI5agxCVu7LHr7JeUE=": 1,
    "https://media.istockphoto.com/id/1216050889/photo/young-woman-with-a-mask-during-pandemic.webp?b=1&s=170667a&w=0&k=20&c=4FY490APpjaBszSyHIpNCsG7N-atU2lFG_h10y3K6wU=": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQZ-8GTOBaELw_3HVApdoD6b7MOccFKsXd3yQ&usqp=CAU": 1,
    "https://www.entandaudiologynews.com/media/20531/ent-covi-mask-convers.jpg?anchor=center&mode=crop&width=750&height=560&rnd=132344394750000000": 1,
    "https://c1.wallpaperflare.com/preview/5/909/303/demonstrator-portrait-fashion-light.jpg": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS7VPVuUMQ-51xJe7FJ3DI2w1RWDNJ_UxLdVA&usqp=CAU": 1,
    "https://articles-1mg.gumlet.io/articles/wp-content/uploads/2020/07/mask.jpg?compress=true&quality=80&w=640&dpr=2.6": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTC28EIEnkZs-oGQ3x-bJ-pO3cTQFjzTVL4BlllpuuuXA&s": 1,
    "https://st2.depositphotos.com/2931363/6569/i/450/depositphotos_65699901-stock-photo-black-man-keeping-arms-crossed.jpg": 0,
    "https://t4.ftcdn.net/jpg/03/83/25/83/360_F_383258331_D8imaEMl8Q3lf7EKU2Pi78Cn0R7KkW9o.jpg": 0,
    "https://t3.ftcdn.net/jpg/02/43/12/34/360_F_243123463_zTooub557xEWABDLk0jJklDyLSGl2jrr.jpg": 0,
    "https://media.hswstatic.com/eyJidWNrZXQiOiJjb250ZW50Lmhzd3N0YXRpYy5jb20iLCJrZXkiOiJnaWZcL3BsYXlcLzBiN2Y0ZTliLWY1OWMtNDAyNC05ZjA2LWIzZGMxMjg1MGFiNy0xOTIwLTEwODAuanBnIiwiZWRpdHMiOnsicmVzaXplIjp7IndpZHRoIjo4Mjh9fX0=": 0,
    "https://img.freepik.com/free-photo/portrait-white-man-isolated_53876-40306.jpg?size=626&ext=jpg&ga=GA1.1.523418798.1710460800&semt=sph": 0,
    "https://drive.google.com/uc?export=view&id=1GkUHKXXaF-StxSy1o3vS0i-m6jzaeWWq": 0,
    "https://cdn.pfizer.com/pfizercom/African%2520businesswoman%2520with%2520face%2520mask%2520in%2520office.jpeg?VersionId=ahc00QsVGdg.In.W5F_sVfGsV4LaEzqz": 1,
    "https://akm-img-a-in.tosshub.com/indiatoday/images/story/202004/Face_Mask.jpeg?size=690:388": 1,
    "https://c8.alamy.com/comp/E6K47F/portrait-indian-man-with-a-white-beard-kerala-south-india-india-E6K47F.jpg": 0,
    "https://drive.google.com/uc?export=view&id=1MMU46F4NkhegcH_aqZbnTHg02s4SAjdQ": 0,
    "https://drive.usercontent.google.com/download?id=1CJT0Qvkuwf7z533MoQLUKIdoyd3CZf2O&export=view&authuser=0": 0,
    "https://drive.google.com/uc?export=view&id=1ijmwglj1F7KZj43Lg1F8jciesE11ONZX": 0,
    "https://drive.google.com/uc?export=download&id=1KxxjeAF3ISxv3Nai9V2YJ5p7-0-Vc0eO": 0,
    "https://drive.google.com/uc?export=view&id=1vQnmrRk2i2fGlpBxtqGOMsUn71ZSx5ca": 1,
    "https://drive.google.com/uc?export=download&id=1oXAZ70i5DMeJP0cMRka1ZRQs5qf4ovnT": 1,
    # Beared Persons 0
    "https://sonsofhollis.com/cdn/shop/articles/Screen_Shot_2021-04-11_at_12.44.09_PM.png?v=1618159467": 0,
    "https://media.istockphoto.com/id/636830292/photo/side-view-portrait-of-thinking-stylish-young-man-looking-away.jpg?s=612x612&w=0&k=20&c=hiacAHKiYZgof4rMzz6d4QXAW7oZUXX2c7pGPjZZ8Uw=": 0,
    "https://www.beardbrand.com/cdn/shop/articles/20190815_Beardbrand_AM_1500x800_2a92409d-12f5-42b2-b246-854ed8332f01_600x600_crop_center.jpg?v=1587764791": 0,
    "https://i.pinimg.com/736x/32/e2/bc/32e2bcb3824b682d8c13bcbbb7cd8d90.jpg": 0,
    "https://qph.cf2.quoracdn.net/main-qimg-6960d1b29a72ec706424b3e739aaca7b-lq": 0,
    # Beared Person 1
    "https://www.eatingwell.com/thmb/T3t5PVwRuh3SAhGaznZwxbxxsuc=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/how-to-correctly-wear-a-mask-if-you-have-a-beard-or-a-moustache-plus-where-to-find-masks-that-fit-right-tk-ea713d189c394d8fa66c2c2aaa2db985.jpg": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcShFam8yWZ7C_p2Wiv7Ntim8gZdq36O8PIiKg&usqp=CAU": 1,
    "https://pyxis.nymag.com/v1/imgs/375/a8a/27c1a1832b622cf301f90dd5dbbea416d4-Yia-Vang-in-Steller-Goods-mask.2x.rvertical.w570.jpg": 1,
    "https://www.harleenkaur.us/cdn/shop/products/FaceMaskforBeard_5d1645e2-8dc8-4f30-9923-8b4747bd7559.jpg?v=1608823643&width=1024": 1,
    # Person with full face mask
    "https://t4.ftcdn.net/jpg/03/55/88/69/360_F_355886955_e29qw53rUhtazD6K7bf2C7UE6uOqmucG.jpg": 1,
    "https://cdn.pixabay.com/photo/2021/06/19/10/01/robber-6348201_640.jpg": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQJPzIb7F8Q98j1bL5fVTS7KFK2CDZfHTGXrw&usqp=CAU": 1,
    "https://i.pinimg.com/474x/e7/27/df/e727df657734f4afea43cf36f2baafa2.jpg": 1,
    "https://t3.ftcdn.net/jpg/04/05/75/56/360_F_405755684_6Grmecqj80nEamzqJBaZpw05Hhz4w84s.jpg": 1,
    # #HackerChief Mask
    "https://media.istockphoto.com/id/1220942331/photo/indian-man-with-handkerchief-mask-for-covid-19-coronavirus-prevention.jpg?s=612x612&w=is&k=20&c=z3NoKjWa2f0At0CIlek3S0WQal4A2QVFTuWnTexVk5g=": 1,
    "https://i.etsystatic.com/5676972/r/il/edac7a/2268292032/il_1080xN.2268292032_gcyq.jpg": 1,
    "https://images.picxy.com/cache/2020/5/21/b448b5c30cfb31a1774ab22a5195566b.jpg": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTCGolpPbq3GJ5JbuC1r2kQuQosc3d9Wgx2GQ&usqp=CAU": 1,
    "https://rukminim2.flixcart.com/image/850/1000/l12h1u80/handkerchief/y/t/u/elon-musk-trending-unisex-cotton-paisley-printed-cowboy-bandanas-original-imagcprgzmzzfnue.jpeg?q=20&crop=false": 1,
    # # Person with side view
    "https://media.istockphoto.com/id/1280516104/photo/selective-focus-of-young-woman-among-many-commuters.jpg?s=612x612&w=0&k=20&c=vF_alg02Od4JRDrREpjTOSym3Et6dMGOYLW1dd93kSg=": 1,
    # FUlly covered Body
    "https://t4.ftcdn.net/jpg/03/59/63/07/360_F_359630745_XvAJtuHsM8qwGYN0tyDWTcGTVKIoXkou.jpg": 1,
    # close Face images
    "https://img.freepik.com/free-photo/beautiful-brunette-serious-girl-portrait_23-2148176376.jpg?size=626&ext=jpg&ga=GA1.1.735520172.1710288000&semt=ais": 0,
    "https://freight.cargo.site/w/1200/i/1d1e402e25938b63b000d64d9639bbd0dce0ccf2066bddd9791667a8d8cc3f4e/MS_West_Kanye_CloseUp.jpg": 0,
    "https://media.istockphoto.com/id/1165314753/photo/born-and-bred-in-the-city.webp?b=1&s=170667a&w=0&k=20&c=6O3CW5WTN_pSu_v80ihw72ev2v1h-x5bK4x8cPNTzxg=": 0,
    "https://img.freepik.com/free-photo/beautiful-woman-face-portrait-close-up-dark_155003-2730.jpg": 0,
    "https://i.pinimg.com/736x/24/c5/65/24c565ead4a0524a1a0a3498687ba487.jpg": 0,
    "https://drive.google.com/uc?export=view&id=1vQnmrRk2i2fGlpBxtqGOMsUn71ZSx5ca": 1,
    "https://drive.google.com/uc?export=view&id=1CJT0Qvkuwf7z533MoQLUKIdoyd3CZf2O": 0,
    "https://drive.google.com/uc?export=view&id=1MMU46F4NkhegcH_aqZbnTHg02s4SAjdQ": 0,
    "https://drive.google.com/uc?export=view&id=1ijmwglj1F7KZj43Lg1F8jciesE11ONZX": 0,
    "https://drive.google.com/uc?export=view&id=1KxxjeAF3ISxv3Nai9V2YJ5p7-0-Vc0eO": 0,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTB-YV9UNl7wwqA1HcScoqwx_xjzvOThFLxdA&usqp=CAU": 1,
    "https://drive.google.com/uc?export=download&id=1oXAZ70i5DMeJP0cMRka1ZRQs5qf4ovnT": 1,
    "https://images.picxy.com/cache/2020/4/12/2e174f18bd32c7197a9386033f14bbd1.jpg": 1,
    "https://t4.ftcdn.net/jpg/03/59/63/07/360_F_359630745_XvAJtuHsM8qwGYN0tyDWTcGTVKIoXkou.jpg": 1,
    "https://c8.alamy.com/comp/E6K47F/portrait-indian-man-with-a-white-beard-kerala-south-india-india-E6K47F.jpg": 0,
    "https://sonsofhollis.com/cdn/shop/articles/Screen_Shot_2021-04-11_at_12.44.09_PM.png?v=1618159467": 0,
    "https://img.freepik.com/free-photo/sideways-black-person-looking-away_23-2148749548.jpg": 0,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS2EMfGVllWUPY89Fn9MCTT2jWmVRGYGBeJMQ&usqp=CAU": 0,
    "https://img.freepik.com/free-photo/sideways-black-person-looking-away_23-2148749548.jpg": 0,
    "https://www.shutterstock.com/image-photo/side-view-young-stylish-man-260nw-602583317.jpg": 0,
    "https://drive.google.com/uc?export=view&id=1GkUHKXXaF-StxSy1o3vS0i-m6jzaeWWq": 0,
    "https://www.eatingwell.com/thmb/T3t5PVwRuh3SAhGaznZwxbxxsuc=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/how-to-correctly-wear-a-mask-if-you-have-a-beard-or-a-moustache-plus-where-to-find-masks-that-fit-right-tk-ea713d189c394d8fa66c2c2aaa2db985.jpg": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcShFam8yWZ7C_p2Wiv7Ntim8gZdq36O8PIiKg&usqp=CAU": 1,
    "https://pyxis.nymag.com/v1/imgs/375/a8a/27c1a1832b622cf301f90dd5dbbea416d4-Yia-Vang-in-Steller-Goods-mask.2x.rvertical.w570.jpg": 1,
    "https://www.harleenkaur.us/cdn/shop/products/FaceMaskforBeard_5d1645e2-8dc8-4f30-9923-8b4747bd7559.jpg?v=1608823643&width=1024": 1,
    "https://t4.ftcdn.net/jpg/03/55/88/69/360_F_355886955_e29qw53rUhtazD6K7bf2C7UE6uOqmucG.jpg": 1,
    "https://cdn.pixabay.com/photo/2021/06/19/10/01/robber-6348201_640.jpg": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQJPzIb7F8Q98j1bL5fVTS7KFK2CDZfHTGXrw&usqp=CAU": 1,
    "https://i.pinimg.com/474x/e7/27/df/e727df657734f4afea43cf36f2baafa2.jpg": 1,
    "https://t3.ftcdn.net/jpg/04/05/75/56/360_F_405755684_6Grmecqj80nEamzqJBaZpw05Hhz4w84s.jpg": 1,
}

<h4>Testing data for mask detection</h4>
<p>This contains unseen data for mask detection model</p>

In [ ]:
Test = {
    "https://media.istockphoto.com/id/1272058180/photo/portrait-of-a-young-man-wearing-protective-face-mask.webp?b=1&s=170667a&w=0&k=20&c=dRWUCyd0VtQg6P15-HsiSlqE1dI5agxCVu7LHr7JeUE=": 1,
    "https://media.istockphoto.com/id/1216050889/photo/young-woman-with-a-mask-during-pandemic.webp?b=1&s=170667a&w=0&k=20&c=4FY490APpjaBszSyHIpNCsG7N-atU2lFG_h10y3K6wU=": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQZ-8GTOBaELw_3HVApdoD6b7MOccFKsXd3yQ&usqp=CAU": 1,
    "https://www.entandaudiologynews.com/media/20531/ent-covi-mask-convers.jpg?anchor=center&mode=crop&width=750&height=560&rnd=132344394750000000": 1,
    "https://c1.wallpaperflare.com/preview/5/909/303/demonstrator-portrait-fashion-light.jpg": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS7VPVuUMQ-51xJe7FJ3DI2w1RWDNJ_UxLdVA&usqp=CAU": 1,
    "https://articles-1mg.gumlet.io/articles/wp-content/uploads/2020/07/mask.jpg?compress=true&quality=80&w=640&dpr=2.6": 1,
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTC28EIEnkZs-oGQ3x-bJ-pO3cTQFjzTVL4BlllpuuuXA&s": 1,
    "https://st2.depositphotos.com/2931363/6569/i/450/depositphotos_65699901-stock-photo-black-man-keeping-arms-crossed.jpg": 0,
    "https://t4.ftcdn.net/jpg/03/83/25/83/360_F_383258331_D8imaEMl8Q3lf7EKU2Pi78Cn0R7KkW9o.jpg": 0,
    "https://t3.ftcdn.net/jpg/02/43/12/34/360_F_243123463_zTooub557xEWABDLk0jJklDyLSGl2jrr.jpg": 0,
    "https://media.hswstatic.com/eyJidWNrZXQiOiJjb250ZW50Lmhzd3N0YXRpYy5jb20iLCJrZXkiOiJnaWZcL3BsYXlcLzBiN2Y0ZTliLWY1OWMtNDAyNC05ZjA2LWIzZGMxMjg1MGFiNy0xOTIwLTEwODAuanBnIiwiZWRpdHMiOnsicmVzaXplIjp7IndpZHRoIjo4Mjh9fX0=": 0,
    "https://img.freepik.com/free-photo/portrait-white-man-isolated_53876-40306.jpg?size=626&ext=jpg&ga=GA1.1.523418798.1710460800&semt=sph": 0,
    "https://drive.google.com/uc?export=view&id=1GkUHKXXaF-StxSy1o3vS0i-m6jzaeWWq": 0,
    "https://cdn.pfizer.com/pfizercom/African%2520businesswoman%2520with%2520face%2520mask%2520in%2520office.jpeg?VersionId=ahc00QsVGdg.In.W5F_sVfGsV4LaEzqz": 1,
    "https://akm-img-a-in.tosshub.com/indiatoday/images/story/202004/Face_Mask.jpeg?size=690:388": 1,
}

<h4>Below map contains training data for authorisation detection model</h4>

In [ ]:
similarDataUrls0 = {
    "https://st2.depositphotos.com/2931363/6569/i/450/depositphotos_65699901-stock-photo-black-man-keeping-arms-crossed.jpg": 0,
    "https://img.freepik.com/free-photo/beautiful-brunette-serious-girl-portrait_23-2148176376.jpg?size=626&ext=jpg&ga=GA1.1.735520172.1710288000&semt=ais": 0,
    "https://freight.cargo.site/w/1200/i/1d1e402e25938b63b000d64d9639bbd0dce0ccf2066bddd9791667a8d8cc3f4e/MS_West_Kanye_CloseUp.jpg": 0,
    "https://media.istockphoto.com/id/1165314753/photo/born-and-bred-in-the-city.webp?b=1&s=170667a&w=0&k=20&c=6O3CW5WTN_pSu_v80ihw72ev2v1h-x5bK4x8cPNTzxg=": 0,
    "https://img.freepik.com/free-photo/beautiful-woman-face-portrait-close-up-dark_155003-2730.jpg": 0,
    "https://i.pinimg.com/736x/24/c5/65/24c565ead4a0524a1a0a3498687ba487.jpg": 0,
    "https://sonsofhollis.com/cdn/shop/articles/Screen_Shot_2021-04-11_at_12.44.09_PM.png?v=1618159467": 0,
    "https://media.istockphoto.com/id/636830292/photo/side-view-portrait-of-thinking-stylish-young-man-looking-away.jpg?s=612x612&w=0&k=20&c=hiacAHKiYZgof4rMzz6d4QXAW7oZUXX2c7pGPjZZ8Uw=": 0,
    "https://www.beardbrand.com/cdn/shop/articles/20190815_Beardbrand_AM_1500x800_2a92409d-12f5-42b2-b246-854ed8332f01_600x600_crop_center.jpg?v=1587764791": 0,
    "https://i.pinimg.com/736x/32/e2/bc/32e2bcb3824b682d8c13bcbbb7cd8d90.jpg": 0,
    "https://qph.cf2.quoracdn.net/main-qimg-6960d1b29a72ec706424b3e739aaca7b-lq": 0,
    "https://img.freepik.com/free-photo/beautiful-brunette-serious-girl-portrait_23-2148176376.jpg?size=626&ext=jpg&ga=GA1.1.735520172.1710288000&semt=ais": 0,
    "https://freight.cargo.site/w/1200/i/1d1e402e25938b63b000d64d9639bbd0dce0ccf2066bddd9791667a8d8cc3f4e/MS_West_Kanye_CloseUp.jpg": 0,
    "https://media.istockphoto.com/id/1165314753/photo/born-and-bred-in-the-city.webp?b=1&s=170667a&w=0&k=20&c=6O3CW5WTN_pSu_v80ihw72ev2v1h-x5bK4x8cPNTzxg=": 0,
    "https://img.freepik.com/free-photo/beautiful-woman-face-portrait-close-up-dark_155003-2730.jpg": 0,
    "https://i.pinimg.com/736x/24/c5/65/24c565ead4a0524a1a0a3498687ba487.jpg": 0,
}

similarDataset={
    "/content/drive/MyDrive/MLCOLAB/DATA1/Screenshot (36).png":1,
    "/content/drive/MyDrive/MLCOLAB/DATA1/WIN_20230525_21_54_18_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/DATA1/WIN_20230526_13_01_42_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/DATA1/WIN_20230526_13_02_21_Pro.jpg":1,

    "/content/drive/MyDrive/MLCOLAB/DATA1/WIN_20230526_13_02_21_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/DATA1/WIN_20230526_13_02_22_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/DATA1/WIN_20240315_23_32_32_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/DATA1/WIN_20240316_12_54_18_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/WIN_20240408_19_59_58_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/WIN_20240408_20_00_05_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/WIN_20240408_20_00_16_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/WIN_20240408_20_00_16_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/WIN_20240408_21_50_10_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/WIN_20240408_21_50_16_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/WIN_20240408_21_50_24_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/WIN_20240408_21_50_26_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/output.png":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/output1.png":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240415_21_28_42_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240415_21_28_48_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240415_21_28_51_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240415_21_28_55_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240415_21_28_58_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240415_22_56_34_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240415_22_56_29_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/output.png":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240415_22_56_24_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240415_22_56_20_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240416_10_10_48_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240416_10_08_07_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240416_10_08_05_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240416_10_08_04_Pro.jpg":1,
    
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240416_11_34_25_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240416_11_34_23_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240416_11_34_20_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240416_11_34_18_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240416_11_34_14_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240416_11_34_12_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240416_11_34_09_Pro.jpg":0,
    

    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.08.41_7be3bef2.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.10.23_603b5087.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.12.13_43353722.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.12.47_b07971f9.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.13.23_29b76960.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.13.57_10240d21.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.14.48_750742c7.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.16.38_452392c2.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.16.46_74a08ce6.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.16.58_39f8b5b4.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA2/WIN_20240409_11_52_40_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA2/WIN_20240409_11_52_45_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA2/WIN_20240409_11_52_55_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA2/WIN_20240409_11_52_57_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA2/WIN_20240409_11_53_01_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA2/WIN_20240409_11_53_06_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA2/WIN_20240409_11_53_10_Pro.jpg":0,
     "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_26_31_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_26_37_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_26_40_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_49_08_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_49_21_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_49_23_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_49_28_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_01_21_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_01_23_Pro.jpg":0,
    # "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_01_25_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_01_33_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_01_40_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_03_00_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_03_11_Pro.jpg":0,
    # "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_03_20_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_03_24_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_03_30_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-15 at 19.36.07_1c323f77.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-15 at 19.36.09_80576e35.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-15 at 19.36.10_7398b5c4.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240415_23_22_40_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240415_23_22_42_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240415_23_22_45_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-15 at 19.36.07_1c323f77.jpg":0,

    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-14 at 13.16.15_491339c3.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-14 at 13.16.17_cc81d6cf.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-14 at 13.16.16_9c3c9565.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-14 at 13.16.15_491339c3.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-14 at 13.15.14_b3bcaca8.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-14 at 13.15.13_100b6400.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-14 at 13.15.12_b11d2a3d.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-14 at 13.15.12_2290a764.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-14 at 13.15.12_76b53d3a.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-14 at 13.15.12_b11d2a3d.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-15 at 20.55.48_4617ec18.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-15 at 20.56.02_eb3d14c3.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-15 at 20.55.38_d626a27f.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WIN_20240415_22_01_15_Pro.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WIN_20240415_22_01_20_Pro.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WIN_20240415_22_01_22_Pro.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WIN_20240415_22_01_24_Pro.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WIN_20240415_22_01_27_Pro.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WIN_20240415_22_01_29_Pro.jpg":2
}

<h4>Below map contains testing data for authorisation detection</h4>

In [ ]:
similarTestData0={
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_26_31_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_26_37_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_26_40_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_03_24_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_22_01_35_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_23_03_00_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_23_02_46_Pro.jpg":0,

    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/WIN_20240408_20_00_16_Pro.jpg":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA0/WIN_20240408_21_26_40_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/output.png":1,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA1/output1.png":1,
     "/content/drive/MyDrive/MLCOLAB/TESTDATA1/Bhavya1.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/TESTDATA2/WIN_20240409_11_53_01_Pro.jpg":2,

    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-15 at 20.56.02_eb3d14c3.jpg":2,
    # "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-15 at 20.55.48_4617ec18.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240415_21_29_02_Pro.jpg":1

    "/content/drive/MyDrive/MLCOLAB/Target3Images/WhatsApp Image 2024-04-15 at 20.56.02_eb3d14c3.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/Target3Images/WIN_20240415_22_01_15_Pro.jpg":2,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-14 at 13.08.41_7be3bef2.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/Target2Images/output.png":1,
  "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-15 at 19.36.07_1c323f77.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-15 at 19.36.09_80576e35.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WhatsApp Image 2024-04-15 at 19.36.10_7398b5c4.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240415_23_22_40_Pro.jpg":0
  "/content/drive/MyDrive/MLCOLAB/Target2Images/WIN_20240416_10_10_51_Pro.jpg":1,
   "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240416_10_08_16_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240416_11_34_18_Pro.jpg":0,
    "/content/drive/MyDrive/MLCOLAB/UnknownImages/WIN_20240416_11_34_09_Pro.jpg":0,

}

<h3>Function to transform image url to pytorch tensors along with data augmentation</h3>

In [ ]:
import requests

def fetch_and_transform_images(urls,store):
    images = []
    labels =[]
    print(urls)
    face_cascade =cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    for url in urls:

        response = requests.get(url)
        img = cv2.imdecode(np.frombuffer(response.content, np.uint8), cv2.IMREAD_COLOR)
        gray =img

        if "gray" not in locals():
          raise ValueError("The 'gray' variable is not defined.")
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        cropped_image=[]
        for (x, y, w, h) in faces:
          cv2.rectangle(gray, (x-30, y-30), (x+w+30, y+h+30), (0, 255, 0), 2)

          cropped_image.append(gray[y:y+h+30,x:x+w+30])

        augmented_images = []

        cropped_image.append(img)
        for close_up in cropped_image:

          close_up=cv2.resize(close_up,(128,128))
          augmented_images.append(close_up)  # Original image

          # Horizontal flip
          augmented_images.append(cv2.flip(close_up, 1))

          # Rotation
          rotated_img = cv2.rotate(close_up, cv2.ROTATE_90_CLOCKWISE)
          augmented_images.append(rotated_img)

          gray_img = cv2.cvtColor(close_up, cv2.COLOR_BGR2GRAY)
          augmented_images.append(cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR))

          low_light_img = np.where((close_up.astype(np.uint16) * 0.5) > 255, 255, (close_up.astype(np.uint16) * 0.5).astype(np.uint8))
          augmented_images.append(low_light_img)

          extra_light_img = np.where((close_up.astype(np.uint16) * 2.0) > 255, 255, (close_up.astype(np.uint16) * 2.0).astype(np.uint8))
          augmented_images.append(extra_light_img)
          # Gaussian blur
          blurred_img = cv2.GaussianBlur(img, (5, 5), 0)
          augmented_images.append(blurred_img)

          images.extend(augmented_images)
        # print(img.shape)
          labels=labels+[store[url]]*len(augmented_images)

    images = torch.from_numpy(np.array(images)).permute(0, 3, 1, 2)  # Permute the tensor to CHW format
    labels=torch.tensor(labels)
    return images,labels


<h3>Function calls to transform image paths to pytorch tensors along with data augmentation</h3>

In [ ]:
def transformImages(paths, store):
    images = []
    labels = []
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    )

    for path in paths:
        img = cv2.imread(path)

        faces = face_cascade.detectMultiScale(img, 1.3, 5)

        cropped_image = []

        for x, y, w, h in faces:
            cv2.rectangle(
                img, (x - 30, y - 30), (x + w + 30, y + h + 30), (0, 255, 0), 2
            )

            cropped_image.append(img[y : y + h + 30, x : x + w + 30])

        augmented_images = []
        for close_up in cropped_image:

            close_up = cv2.resize(close_up, (128, 128))
            augmented_images.append(close_up)  # Original image

            # Horizontal flip
            augmented_images.append(cv2.flip(close_up, 1))

            #   Rotation
            rotated_img = cv2.rotate(close_up, cv2.ROTATE_90_CLOCKWISE)
            augmented_images.append(rotated_img)

            gray_img = cv2.cvtColor(close_up, cv2.COLOR_BGR2GRAY)
            augmented_images.append(cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR))

            low_light_img = np.where(
                (close_up.astype(np.uint16) * 0.5) > 255,
                255,
                (close_up.astype(np.uint16) * 0.5).astype(np.uint8),
            )
            augmented_images.append(low_light_img)

            extra_light_img = np.where(
                (close_up.astype(np.uint16) * 2.0) > 255,
                255,
                (close_up.astype(np.uint16) * 2.0).astype(np.uint8),
            )
            augmented_images.append(extra_light_img)
            # Gaussian blur
            blurred_img = cv2.GaussianBlur(img, (5, 5), 0)
            augmented_images.append(blurred_img)

            images.extend(augmented_images)
            labels = labels + [store[path]] * len(augmented_images)
    images = torch.from_numpy(np.array(images)).permute(
        0, 3, 1, 2
    )  # Permute the tensor to CHW format
    labels = torch.tensor(labels)
    return images, labels

<h3>Function Calls to transform image paths and urls to pytorch tensors </h3> 

<h4>Transformation for masked dataset</h4>

In [ ]:
MaskedTrainImages,MaskedTrainLabels=fetch_and_transform_images(list(data.keys()),data)
MaskedTestImages,MaskedTestLabels=fetch_and_transform_images(list(Test.keys()),Test)

<h4>The below cell is for authorisation model</h4>

In [ ]:
similarTrainImages,TrainLabels=transformImages(list(similarDataset.keys()),similarDataset)


<h3>Creating CNN model class for mask detection</h3>
<p> It is made up of 3 block with input nodes,hidden nodes and output nodes as parameters</p>


In [2]:
class MaskDetectionModel(nn.Module):
  def __init__(self,input_nodes:int,output_nodes:int,hidden_nodes:int):
    super().__init__()

    self.Block1=nn.Sequential(
        nn.Conv2d(in_channels=input_nodes,out_channels=hidden_nodes,kernel_size=2,padding=0,stride=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_nodes,out_channels=hidden_nodes,kernel_size=2,padding=0,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
      )
    self.Block2=nn.Sequential(
        nn.Conv2d(in_channels=hidden_nodes,out_channels=hidden_nodes,kernel_size=2,padding=0,stride=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_nodes,out_channels=hidden_nodes,kernel_size=2,padding=0,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.Block3=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_nodes*900,out_features=1),
        nn.Sigmoid()
    )
  def forward(self,x):
    x=self.Block1(x)
    x=self.Block2(x)
    x=self.Block3(x)
    return x



<h3>Creating CNN model class for similar image detection</h3>
<p> It is made up of 3 block with input nodes,hidden nodes and output nodes as parameters</p>

In [3]:
class SimilarImageModel(nn.Module):
  def __init__(self,input_nodes:int,output_nodes:int,hidden_nodes:int):
    super().__init__()

    self.Block1=nn.Sequential(
        nn.Conv2d(in_channels=input_nodes,out_channels=hidden_nodes,kernel_size=2,padding=0,stride=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_nodes,out_channels=hidden_nodes,kernel_size=2,padding=0,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
      )
    self.Block2=nn.Sequential(
        nn.Conv2d(in_channels=hidden_nodes,out_channels=hidden_nodes,kernel_size=2,padding=0,stride=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_nodes,out_channels=hidden_nodes,kernel_size=2,padding=0,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.Block3=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_nodes*900,out_features=output_nodes),

    )
  def forward(self,x):
    x=self.Block1(x)
    x=self.Block2(x)
    # print(x.shape)
    x=self.Block3(x)
    return x

<h3>Function to load the model from google drive </h3>
<p> The model is loaded from google drive , make sure the model is in the same folder and authorisation is complete </p>

In [4]:
def load_model():
    model = MaskDetectionModel(input_nodes=3, output_nodes=1, hidden_nodes=16)

    model.load_state_dict(torch.load('./trainedmodel.pth'))
    model.eval()  # Set the model to evaluation mode
    return model
def load_similar_model():
    model = SimilarImageModel(input_nodes=3, output_nodes=3, hidden_nodes=40)

    model.load_state_dict(torch.load('./trainedsimilarmodel3classes.pth'))
    # Ensure the model is on GPU if available
    # if torch.cuda.is_available():
    #     model = model.cuda()
    model.eval()  # Set the model to evaluation mode
    return model

<h3>Function calls to load model </h3>

In [5]:
model=load_model()
similarModel=load_similar_model()

<h3>Training loop and hyperparameters for mask detection </h3>

In [90]:

LEARNING_RATE=0.0001

loss_fun1=nn.BCELoss()

optimizer=torch.optim.SGD(model.parameters(),lr=LEARNING_RATE)

In [ ]:
Loss=0
for epoch in range(10):
  model.train()
  optimizer.zero_grad()
  output=model(MaskedTrainImages.float())
  loss=loss_fun1(output.squeeze(dim=1).float(),MaskedTrainLabels.float())
  Loss=loss
  loss.backward()
  optimizer.step()
  if epoch%10==0:
    print( 'Loss:', loss)
  print( 'Loss:', loss)
print( 'Final Loss:', loss)

<h3>Training loop and hyperparameters for similar image detection </h3>

In [ ]:
LEARNING_RATE=0.001

loss_fun1=nn.CrossEntropyLoss()

optimizer=torch.optim.SGD(similarModel.parameters(),lr=LEARNING_RATE)

In [ ]:
Loss=0
for epoch in range(10):
  similarModel.train()
  optimizer.zero_grad()
  output=similarModel(similarTrainImages.squeeze().float())
  output=torch.softmax(output, dim=1)
  # print(output)
  loss=loss_fun1(output,TrainLabels)
  Loss=loss
  loss.backward()
  optimizer.step()
  if epoch%10==0:
    print( 'Loss:', loss)
  print( 'Loss:', loss)
print( 'Loss:', loss)



In [ ]:
testImages,TestLabels=transformImages(list(similarTestData0.keys()),similarTestData0)


In [6]:
with torch.inference_mode():
  y_logits=similarModel(testImages.float())
  # print(y_logits)
  y_prob = torch.softmax(y_logits, dim=1)
  y_pred = torch.argmax(y_prob, dim=1)
  print(y_pred,y_prob)

<h3>For visualisation of Output</h3>
<p>In this section we will be visualising the output using matplotlib</p>
<p>The programe will plot images along with their predicted output and actual output</p>

In [ ]:
num_images = (len(testImages) // 3) + (len(testImages) % 3)
for i in range(num_images):
  start_idx = i * 3
  end_idx = (i + 1) * 3
  fig, axs = plt.subplots(1, 3, figsize=(15, 5))
  for j in range(3):
    idx = start_idx + j
    if idx>=len(testImages):
      print("hi")
      break
    axs[j].imshow(testImages[idx].permute(1, 2, 0))
    axs[j].set_title(f"Actual: {TestLabels[idx].item()}, Predicted: {y_pred[idx].item():.2f}")
    axs[j].axis('off')
  plt.show()


<h3>Confusion matrix for model</h3>

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

y_pred = (y_logits >= threshold).float()


conf_matrix = confusion_matrix(TestLabels, y_pred)

precision = precision_score(TestLabels, y_pred)
recall = recall_score(TestLabels, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("Precision:", precision)
print("Recall:", recall)

<h3>Using model for live detection</h3>

In [21]:

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

gun_cascade = cv2.CascadeClassifier('./cascade.xml')

cap = cv2.VideoCapture(0) 

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/content/output_video.avi', fourcc, 30.0, (640, 480))  # Adjust resolution if needed


while True:

    ret, frame = cap.read()


    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    gun=gun_cascade.detectMultiScale(gray_frame,1.3,20,minSize=(50,50))
    for  (x, y, w, h) in gun:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    

    for (x, y, w, h) in faces:
        # Extract the detected face
        face = frame[max(0,y-60):y+h+60, max(0,x-60):x+w+60]

        # Convert the face to the desired size and format
        face_np = cv2.resize(face, (128, 128), interpolation=cv2.INTER_AREA)
        face_tensor = torch.tensor(face_np)
        face_tensor = face_tensor.permute(2, 0, 1)
        face_tensor = face_tensor.unsqueeze(0)

        # Perform inference with your model
        model.eval()
        similarModel.eval()
        with torch.inference_mode():
            y_logits = model(face_tensor.float())
            
            threshold = 0.59
            pred = (y_logits >= threshold).float()
            if pred == 1:
               label=f' Mask: {y_logits.item():.2f}'
            else:
                isKnown_logits=similarModel(face_tensor.float())
                isKnown_prob=torch.softmax(isKnown_logits,dim=1)
                isKnown_pred=isKnown_prob.argmax(dim=1)
                if isKnown_pred==1:
                    label=f'Ankit No Mask: {y_logits.item():.2f}'
                elif isKnown_pred==2 :
                    label=f'Nisarg No Mask:: {y_logits.item():.2f}'
                else:
                    label=f'unknown person No Mask: {y_logits.item():.2f}'
            cv2.putText(frame, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

        # Draw rectangles around the detected faces
        cv2.rectangle(frame, (x-20, y-60), (x+w+60, y+h+60), (0, 255, 0), 2)

    # Write the frame with rectangles into the output video
    out.write(frame)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object, the video writer object, and close all OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()


<h3>Using Model with prerecorded video</h3>

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

gun_cascade = cv2.CascadeClassifier('./cascade.xml')

# Open the video file
cap = cv2.VideoCapture('/path/to/video/file.mp4')  

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/content/output_video_path.avi', fourcc, fps, (int(cap.get(3)), int(cap.get(4))))



# Process each frame of the video
while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break


    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    gun=gun_cascade.detectMultiScale(gray_frame,1.3,20,minSize=(50,50))
    for  (x, y, w, h) in gun:
      cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    # Draw rectangles around the detected faces and add labels
    for (x, y, w, h) in faces:
        # Extract the detected face
        face = frame[max(0,y-60):y+h+60, max(0,x-60):x+w+60]

        # Convert the face to the desired size and format
        face_np = cv2.resize(face, (128, 128), interpolation=cv2.INTER_AREA)
        face_tensor = torch.tensor(face_np)
        face_tensor = face_tensor.permute(2, 0, 1)
        face_tensor = face_tensor.unsqueeze(0)

        # Perform inference with your model
        with torch.inference_mode():
            y_logits = model(face_tensor.float())
            threshold = 0.59
            pred = (y_logits >= threshold).float()
            if pred == 1:
               label=f'Mask: {y_logits.item():.2f}'
            else:
                isKnown_logits=similarModel(face_tensor.float())
                isKnown_prob=torch.softmax(isKnown_logits,dim=1)
                isKnown_pred=isKnown_prob.argmax(dim=1)
                print(isKnown_pred,isKnown_prob)
                if isKnown_pred==1:
                    label=f'Ankit No Mask: {y_logits.item():.2f}'
                # cv2.putText(frame, 'No Mask', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                else:
                    label=f'unknown person No Mask: {y_logits.item():.2f}'
            cv2.putText(frame, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

        # Draw rectangles around the detected faces
        cv2.rectangle(frame, (x-60, y-120), (x+w+60, y+h+60), (0, 255, 0), 2)

    # Write the frame with rectangles into the output video
    out.write(frame)

# Release the video capture object, the video writer object, and close all OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()
